In [31]:
import numpy as np

dataset_input = np.load('D:/Dataset_1/input.npy', allow_pickle=True) 
dataset_output = np.load('D:/Dataset_1/output.npy', allow_pickle=True)

dataset_input = dataset_input.astype('float32') / 10.
dataset_output  = dataset_output.astype('float32') / 10.
print(dataset_input)
print(dataset_output)

[[0.3 0.3 0.6 ... 0.4 0.1 0.6]
 [0.1 0.2 0.2 ... 0.3 0.6 0.3]
 [0.3 0.8 0.3 ... 0.1 0.1 0.3]
 ...
 [0.1 0.8 0.1 ... 0.1 0.6 0.2]
 [0.3 0.1 0.3 ... 0.9 0.2 0.9]
 [0.2 0.1 0.3 ... 0.3 0.1 0.2]]
[[0.3 0.3 0.  ... 0.9 0.4 0.7]
 [0.1 0.2 0.  ... 0.4 0.6 0.7]
 [0.3 0.8 0.  ... 0.1 0.7 0.7]
 ...
 [0.1 0.8 0.  ... 0.4 0.8 0.7]
 [0.3 0.1 0.  ... 0.8 0.1 0.7]
 [0.2 0.1 0.  ... 0.9 0.7 0.7]]


In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset_input, dataset_output, test_size=0.2, random_state=1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1600, 200)
(1600, 500)
(400, 200)
(400, 500)


In [33]:
type(X_train)

numpy.ndarray

In [23]:
from keras.layers import Input, Dense
from keras.models import Model

def create_dense_ae():
    # Dimension of coded representation
    encoding_dim = 200

    # Encoder
    # Input placeholder
    input_element = Input(shape=200) # 1, 200, 1 - dimensions of strings, cloumns and filters of one sample element
    # Representation coded with fully connected layer
    encoded = Dense(encoding_dim, activation='relu')(input_element)
    
    # Decoder
    # Representation uncoded with another fully connected layer
    input_encoded = Input(shape=(encoding_dim,))
    decoded = Dense(1*500, activation='sigmoid')(input_encoded)

    # Model(input_layer, output_layer)
    # Model(model_1, model_2)
    encoder = Model(input_element, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_element, decoder(encoder(input_element)), name="autoencoder")
    return encoder, decoder, autoencoder

In [24]:
encoder, decoder, autoencoder = create_dense_ae()
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [25]:
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
encoder (Functional)         (None, 200)               40200     
_________________________________________________________________
decoder (Functional)         (None, 500)               100500    
Total params: 140,700
Trainable params: 140,700
Non-trainable params: 0
_________________________________________________________________


In [39]:
print(y_test[0])

[0.1 0.2 0.  0.  0.  0.5 0.1 0.1 0.  0.  0.5 0.3 0.  0.2 0.  0.1 0.4 0.5
 0.2 0.  0.3 0.5 0.6 0.  0.  0.1 0.4 0.9 0.  0.  0.1 0.1 0.6 0.5 0.  0.1
 0.1 0.9 0.5 0.  0.1 0.2 0.  0.6 0.  0.1 0.2 0.1 0.6 0.  0.1 0.2 0.  0.8
 0.  0.1 0.2 0.1 0.8 0.  0.3 0.3 0.2 0.6 0.  0.4 0.3 0.5 0.6 0.  0.4 0.1
 0.2 0.9 0.  0.3 0.1 0.6 0.9 0.  0.6 0.2 0.  0.  0.1 0.5 0.6 0.  0.3 0.1
 0.2 0.1 0.6 0.  0.1 0.2 0.7 0.6 0.2 0.1 0.7 0.1 0.  0.9 0.1 0.1 0.1 0.7
 0.9 0.1 0.1 0.1 0.8 0.  0.1 0.1 0.1 0.9 0.  0.1 0.1 0.1 0.8 0.1 0.1 0.1
 0.6 0.8 0.3 0.1 0.1 0.1 0.9 0.1 0.1 0.1 0.8 0.9 0.2 0.1 0.1 0.4 0.  0.
 0.2 0.1 0.3 0.  0.4 0.2 0.1 0.1 0.1 0.  0.2 0.1 0.5 0.1 0.2 0.2 0.1 0.1
 0.  0.7 0.2 0.1 0.1 0.  0.9 0.2 0.1 0.1 0.1 0.7 0.2 0.1 0.1 0.1 0.8 0.2
 0.1 0.4 0.2 0.  0.2 0.1 0.1 0.2 0.5 0.2 0.6 0.3 0.4 0.  0.2 0.4 0.2 0.4
 0.3 0.2 0.7 0.3 0.2 0.6 0.2 0.7 0.1 0.2 0.9 0.2 0.1 0.2 0.9 0.6 0.2 0.1
 0.1 0.9 0.8 0.2 0.4 0.1 0.  0.  0.3 0.4 0.6 0.  0.1 0.3 0.2 0.1 0.4 0.
 0.3 0.3 0.4 0.4 0.3 0.3 0.4 0.1 0.  0.7 0.3 0.4 0.2 

In [34]:
autoencoder.fit(X_train, y_train,
                epochs=50,
                batch_size=100,
                shuffle=False,
                validation_data=(X_test, y_test))

Epoch 1/50
16/16 [==============================] - 0s 12ms/step - loss: 11098.9697 - val_loss: 11145.3867
Epoch 2/50
16/16 [==============================] - 0s 6ms/step - loss: 11181.9941 - val_loss: 11160.4648
Epoch 3/50
16/16 [==============================] - 0s 6ms/step - loss: 11178.2305 - val_loss: 11145.0635
Epoch 4/50
16/16 [==============================] - 0s 5ms/step - loss: 11160.2051 - val_loss: 11124.9775
Epoch 5/50
16/16 [==============================] - 0s 5ms/step - loss: 11139.3350 - val_loss: 11103.6016
Epoch 6/50
16/16 [==============================] - 0s 6ms/step - loss: 11117.8584 - val_loss: 11082.0723
Epoch 7/50
16/16 [==============================] - 0s 6ms/step - loss: 11096.2197 - val_loss: 11060.3496
Epoch 8/50
16/16 [==============================] - 0s 6ms/step - loss: 11074.4072 - val_loss: 11038.4678
Epoch 9/50
16/16 [==============================] - 0s 5ms/step - loss: 11052.4502 - val_loss: 11016.4570
Epoch 10/50
16/16 [==========================

In [40]:
n = 1

#imgs = X_test[:n]
imgs = X_train[:n]
encoded = encoder.predict(imgs, batch_size=n)

#decoded = np.round(decoder.predict(encoded, batch_size=n), 0)
decoded = decoder.predict(encoded, batch_size=n)
print(decoded[0])
print(y_test[0])

[1.         1.         0.01432377 0.02253425 0.         1.
 1.         1.         1.         0.         1.         1.
 1.         1.         0.         1.         1.         1.
 1.         0.         1.         1.         1.         1.
 0.         1.         1.         1.         1.         0.
 1.         1.         1.         1.         0.         1.
 1.         1.         1.         0.         1.         1.
 1.         1.         0.00440159 1.         1.         1.
 1.         0.00493029 1.         1.         1.         1.
 0.01982111 1.         1.         1.         1.         0.02341294
 1.         1.         1.         1.         0.02294454 1.
 1.         1.         1.         0.02377334 1.         1.
 1.         1.         0.05179331 1.         1.         1.
 1.         0.05493581 1.         1.         1.         1.
 0.08868879 1.         1.         1.         1.         0.09992495
 1.         1.         1.         1.         0.1057457  1.
 1.         1.         1.         0.0811

In [41]:
def create_deep_dense_ae():
    # Размерность кодированного представления
    encoding_dim = 200

    # Энкодер
    input_img = Input(shape=200)
    x = Dense(encoding_dim*1, activation='relu')(input_img)
    x = Dense(encoding_dim*2, activation='relu')(x)
    encoded = Dense(encoding_dim, activation='linear')(x)
    
    # Декодер
    input_encoded = Input(shape=(encoding_dim,))
    x = Dense(encoding_dim*2, activation='relu')(input_encoded)
    x = Dense(encoding_dim*3, activation='relu')(x)
    decoded = Dense(1*500, activation='sigmoid')(x)
    
    # Модели
    encoder = Model(input_img, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")
    return encoder, decoder, autoencoder

d_encoder, d_decoder, d_autoencoder = create_deep_dense_ae()
d_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [42]:
d_autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 200)]             0         
_________________________________________________________________
encoder (Functional)         (None, 200)               200800    
_________________________________________________________________
decoder (Functional)         (None, 500)               621500    
Total params: 822,300
Trainable params: 822,300
Non-trainable params: 0
_________________________________________________________________


In [43]:
d_autoencoder.fit(X_train, y_train,
                epochs=50,
                batch_size=100,
                shuffle=False,
                validation_data=(X_test, y_test))

Epoch 1/50
16/16 [==============================] - 1s 35ms/step - loss: 0.6183 - val_loss: 0.5865
Epoch 2/50
16/16 [==============================] - 0s 16ms/step - loss: 0.5845 - val_loss: 0.5825
Epoch 3/50
16/16 [==============================] - 0s 15ms/step - loss: 0.5826 - val_loss: 0.5815
Epoch 4/50
16/16 [==============================] - 0s 15ms/step - loss: 0.5820 - val_loss: 0.5811
Epoch 5/50
16/16 [==============================] - 0s 14ms/step - loss: 0.5815 - val_loss: 0.5806
Epoch 6/50
16/16 [==============================] - 0s 15ms/step - loss: 0.5806 - val_loss: 0.5794
Epoch 7/50
16/16 [==============================] - 0s 15ms/step - loss: 0.5790 - val_loss: 0.5777
Epoch 8/50
16/16 [==============================] - 0s 15ms/step - loss: 0.5771 - val_loss: 0.5764
Epoch 9/50
16/16 [==============================] - 0s 20ms/step - loss: 0.5751 - val_loss: 0.5747
Epoch 10/50
16/16 [==============================] - 0s 16ms/step - loss: 0.5728 - val_loss: 0.5727
Epoch 11/

In [44]:
n = 1

#imgs = X_test[:n]
imgs = X_train[:n]
encoded = encoder.predict(imgs, batch_size=n)

#decoded = np.round(decoder.predict(encoded, batch_size=n), 0)
decoded = decoder.predict(encoded, batch_size=n)
print(decoded[0])
print(y_test[0])

[1.         1.         0.01432377 0.02253425 0.         1.
 1.         1.         1.         0.         1.         1.
 1.         1.         0.         1.         1.         1.
 1.         0.         1.         1.         1.         1.
 0.         1.         1.         1.         1.         0.
 1.         1.         1.         1.         0.         1.
 1.         1.         1.         0.         1.         1.
 1.         1.         0.00440159 1.         1.         1.
 1.         0.00493029 1.         1.         1.         1.
 0.01982111 1.         1.         1.         1.         0.02341294
 1.         1.         1.         1.         0.02294454 1.
 1.         1.         1.         0.02377334 1.         1.
 1.         1.         0.05179331 1.         1.         1.
 1.         0.05493581 1.         1.         1.         1.
 0.08868879 1.         1.         1.         1.         0.09992495
 1.         1.         1.         1.         0.1057457  1.
 1.         1.         1.         0.0811